In [2]:
!pip install beautifulsoup4
!pip install urllib3

In [107]:
import urllib.request
from bs4 import BeautifulSoup

url = 'https://minecraftitemids.com/'
pages = [str(n) for n in range(9)]

def get_content(url):
  req = urllib.request.Request(url)
  html = urllib.request.urlopen(req)
  soup = BeautifulSoup(html, "html.parser")
  topicsindex = soup.find('table', attrs={'id': 'table'})
  topics = topicsindex.find_all('tr')
  return topics

In [106]:
[str(n) for n in range(9)]

['0', '1', '2', '3', '4', '5', '6', '7', '8']

In [71]:
def get_id(line):
  blockID = line.split(',')[0]
  blockID = blockID.split(': ')[1].strip().strip("'")
  return blockID

def get_data(line):
  if "," in line:
    blockData = line.split(',')[0]
  else:
    blockData = line
  blockData = blockData.split(': ')[1].strip().strip("'")
  return blockData

In [109]:
# 1. block_info.jsを走査してblockIDとblockDataを取得
# 2. 読み込み済みのhtmlから対象のblockIDとblockDataと同じ情報を持つ要素を取得
# 3. 抽出した要素のItem IDを取得

from tqdm import tqdm
import pandas as pd

col = ['blockID','blockData','itemID']
item_df = pd.DataFrame(columns=col)
def store_df(item_df,topics):
  with open('./block_info.js') as f:
    while True:
      s_line = f.readline()

      # 最後
      if not s_line:
        break

      blockID = None
      blockData = None
      if 'blockID' in s_line:
        blockID = get_id(s_line)
        data_line = f.readline()
        blockData = get_data(data_line)
      elif 'blockData' in s_line:
        blockData = get_data(s_line)
        id_line = f.readline()
        blockID = get_id(id_line)

      if blockID == None and blockData == None:
        continue

      # print(blockID , blockData)

      for topic in topics:
        tds = topic.find_all('td')
        # header
        if len(tds) <= 4:
          continue
        if ':' in tds[4].text:
          if tds[4].text.split(':')[0] == blockID and tds[4].text.split(':')[1] == blockData:
            item_df = item_df.append(pd.DataFrame([[blockID,blockData,tds[2].text]],columns=col))
            break
        else:
          if tds[4].text == blockID and '0' == blockData:
            item_df = item_df.append(pd.DataFrame([[blockID,'-',tds[2].text]],columns=col))
            break
  return item_df

In [105]:
item_df

,blockID,blockData,itemID
0,1,-,minecraft:stone
0,1,1,minecraft:granite
0,1,2,minecraft:polished_granite
0,1,3,minecraft:diorite
0,1,4,minecraft:polished_diorite
...,...,...,...
0,33,-,minecraft:piston
0,29,-,minecraft:sticky_piston
0,46,-,minecraft:tnt
0,27,-,minecraft:powered_rail


In [110]:
url = 'https://minecraftitemids.com/'
pages = [str(n) for n in range(9)]

col = ['blockID','blockData','itemID']
item_df = pd.DataFrame(columns=col)

for page in pages:
  page_url = url + page
  topics = get_content(page_url)
  item_df = store_df(item_df,topics)

In [111]:
item_df

,blockID,blockData,itemID
0,1,-,minecraft:stone
0,1,1,minecraft:granite
0,1,2,minecraft:polished_granite
0,1,3,minecraft:diorite
0,1,4,minecraft:polished_diorite
...,...,...,...
0,449,-,minecraft:totem_of_undying
0,439,-,minecraft:spectral_arrow
0,396,-,minecraft:golden_carrot
0,414,-,minecraft:rabbit_foot


In [113]:
item_df.to_csv('./item_df.csv',index=False)

## block_info.jsに情報書き込み

In [133]:
insert_rows = []
for row in item_df.itertuples():
  blockID = row.blockID
  blockData = row.blockData if row.blockData != '-' else '0'
  itemID = row.itemID
  
  with open('./block_info.js') as f:
    all_lines = f.readlines()
    blockID_indexes = [i for i,line in enumerate(all_lines) if 'blockID: ' + blockID in line]
    blockID_indexes.extend([i for i,line in enumerate(all_lines) if "blockID: '" + blockID + "'" in line])
    blockData_indexes = [i for i,line in enumerate(all_lines) if 'blockData: ' + blockData in line]
    blockData_indexes.extend([i for i,line in enumerate(all_lines) if "blockData: '" + blockData + "'" in line])

    # insertする行を特定する
    finish = False
    for id in blockID_indexes:
      for data in blockData_indexes:
        if int(data) == int(id) -1 or int(data) == int(id) + 1:
          insert_rows.append([min(int(data),int(id)) - 1,itemID])
          finish = True
          break
      if finish == True:
        break

insert_rows = sorted(insert_rows)

In [136]:
def get_unique_list(seq):
  seen = []
  return [x for x in seq if x not in seen and not seen.append(x)]

insert_rows = get_unique_list(insert_rows)
insert_rows

[[16, 'minecraft:granite'],
 [24, 'minecraft:polished_granite'],
 [32, 'minecraft:diorite'],
 [40, 'minecraft:polished_diorite'],
 [48, 'minecraft:andesite'],
 [56, 'minecraft:polished_andesite'],
 [80, 'minecraft:coarse_dirt'],
 [88, 'minecraft:podzol'],
 [104, 'minecraft:oak_planks'],
 [112, 'minecraft:spruce_planks'],
 [120, 'minecraft:birch_planks'],
 [128, 'minecraft:jungle_planks'],
 [136, 'minecraft:acacia_planks'],
 [144, 'minecraft:dark_oak_planks'],
 [168, 'minecraft:red_sand'],
 [208, 'minecraft:oak_log'],
 [216, 'minecraft:spruce_log'],
 [224, 'minecraft:birch_log'],
 [232, 'minecraft:jungle_log'],
 [240, 'minecraft:sponge'],
 [248, 'minecraft:wet_sponge'],
 [256, 'minecraft:glass'],
 [272, 'minecraft:lapis_block'],
 [280, 'minecraft:sandstone'],
 [288, 'minecraft:chiseled_sandstone'],
 [296, 'minecraft:cut_sandstone'],
 [312, 'minecraft:orange_wool'],
 [320, 'minecraft:magenta_wool'],
 [328, 'minecraft:light_blue_wool'],
 [336, 'minecraft:yellow_wool'],
 [344, 'minecraft:l

In [139]:
with open('./block_info.js') as f:
  contents = f.readlines()

insert_count = 0
for insert_row , itemID in insert_rows:
  insert_content = "itemID: '" + itemID + "',\n"
  contents.insert(insert_row + insert_count + 1,insert_content)
  insert_count += 1

with open('./block_info_mod.js', mode='w') as f:
    f.writelines(contents)